In [0]:
%fs cp dbfs:/FileStore/tables/neon_key_config.json  file:/neon/dh/config/jmh/neon_key_config.json

In [0]:
%fs cp dbfs:/FileStore/tables/neon_jmh_location_config.json  file:/neon/dh/config/jmh/neon_jmh_location_config.json

In [0]:
%fs cp dbfs:/FileStore/tables/neon_jmh_schema_config.json  file:/neon/dh/config/jmh/neon_jmh_schema_config.json

In [0]:
%fs cp dbfs:/FileStore/tables/neon_cosmos_config.json  file:/neon/dh/config/jmh/neon_cosmos_config.json

In [0]:
%fs cp dbfs:/FileStore/tables/neon_jmh_customer_dq_config.json  file:/neon/dh/config/jmh/neon_jmh_customer_dq_config.json

In [0]:
%run "../../../../Common/Utils"  $config_file_location="/neon/dh/config/jmh/" $key_config_file="neon_key_config.json" $location_config_file="neon_jmh_location_config.json" $schema_config_file="neon_jmh_schema_config.json" $cosmos_config_file="neon_cosmos_config.json"

In [0]:
### Setting the Spark Environment
spark.conf.set("fs.azure.account.key."+landing_zone+".blob.core.windows.net",landing_zone_storage_account_access_key)
spark.conf.set("fs.azure.account.key."+curated_zone+".blob.core.windows.net",curated_zone_storage_account_access_key)

In [0]:
###Creating the Schema for Different Source DataFrame
jmh_customer = Create_Schema(create_schema_tuple("Jumeriah_Customer"))

In [0]:
####Getting the CosmosConfig for Write Invalid Records
writeConfig = Invalid_writeConfig()

In [0]:
%run "../../../../Common/DataQuality_FrameWork" $config_file_location="/neon/dh/config/jmh/" $dq_config_file="neon_jmh_customer_dq_config.json"

In [0]:
####Reading the SourceFiles
date_in = datetime.datetime.now().strftime("%Y%m%d")
date_folder = datetime.datetime.now().strftime("%Y/%m/%d")+'/'
#sourcefilelocation = "wasbs://"+event_type+"@"+landing_zone+".blob.core.windows.net"+'/'+date_folder
sourcefilelocation = "wasbs://"+event_type+"@"+landing_zone+".blob.core.windows.net"+'/neonprodjmhlanding/'+"2019/02/15/"
#file_name = "ociscustomer_"+date_in+".csv"
file_name = "ocis_customers_dump.csv"
try:
  jmh_customer_df = Read_File_Format(sourcefilelocation,file_name,jmh_customer,'^').replace('',None)
except:
  message = file_name + " is not present in " + sourcefilelocation
  raise Exception(message)
#jmh_customer_df.count()

In [0]:
# invalid_records_blob_location ="wasbs://"+event_type+"@"+curated_zone+".blob.core.windows.net"+"/jmh_invalid_customerhub_customer/"+date_folder
# valid_records_blob_location = "wasbs://"+event_type+"@"+curated_zone+".blob.core.windows.net"+"/jmh_valid_customerhub_customer/"+date_folder
#invalid_records_blob_location ="wasbs://"+event_type+"@"+curated_zone+".blob.core.windows.net"+"/neonprodjmhcurated/jmh_invalid_customerhub_customer/"+date_folder
#valid_records_blob_location = "wasbs://"+event_type+"@"+curated_zone+".blob.core.windows.net"+"/neonprodjmhcurated/jmh_valid_customerhub_customer/"+date_folder
invalid_records_blob_location ="wasbs://"+event_type+"@"+curated_zone+".blob.core.windows.net"+"/neonprodjmhcurated/jmh_invalid_customerhub_customer/"+"2019/02/15/"
invalid_records_blob_location

In [0]:
file_path = 'wasbs://neon-prod-jumeirah@neonprodstorage.blob.core.windows.net/neonprodjmhcurated/jmh_invalid_customerhub_customer/2019/02/15/'
invalid_customer = Read_File_Format_Without_Schema(file_path,"part*.json")
invalid_customer.registerTempTable('invalid_customer')

In [0]:
%sql
select count(*) from invalid_customer

In [0]:
####Write Invalid Numeric Records of Customer Details to Error File
jmh_customer_numeric_validation_cols = dq_config['jmh_customer_numeric_validation_cols']
jmh_invalid_numeric_df = Get_Invalid_Numeric_Records(jmh_customer_df,jmh_customer_numeric_validation_cols).withColumn("vertical",lit("Jumeirah"))
#Write_DataFrame_To_Blob(jmh_invalid_numeric_df,invalid_records_blob_location)
#display(jmh_invalid_numeric_df)
####Write Invalid Numeric Records of Customer Details to CosmosDB File
#jmh_invalid_numeric_df.write.format("com.microsoft.azure.cosmosdb.spark").mode("append").options(**writeConfig).save()
# jmh_invalid_numeric_df.count()
jmh_invalid_numeric_df.registerTempTable('jmh_invalid_numeric_df')

In [0]:
%sql
select Error_desc,count(*) from jmh_invalid_numeric_df group by Error_desc

In [0]:
###Write Invalid DateFormat Records of Customer Details to Error File
jmh_date_validation_cols = dq_config['jmh_customer_date_validation_cols']
jmh_invalid_date_df = Get_Invalid_DateFormat_Records(jmh_customer_df,jmh_date_validation_cols).withColumn("vertical",lit("Jumeirah"))
Write_DataFrame_To_Blob(jmh_invalid_date_df,invalid_records_blob_location,'append')
#display(jmh_invalid_date_df)
####Write Invalid DateFormat Records of Customer Details to CosmosDB File
#jmh_invalid_date_df.write.format("com.microsoft.azure.cosmosdb.spark").mode("append").options(**writeConfig).save()
jmh_invalid_date_df.registerTempTable('jmh_invalid_date_df')

In [0]:
%sql
select Error_desc,count(*) from jmh_invalid_date_df group by Error_desc

In [0]:
###Write Invalid email Format Records of Customer Details to Error File
jmh_email_validation_cols = dq_config['jmh_customer_email_validation_cols']
jmh_invalid_email_df = Get_Invalid_Email_Records(jmh_customer_df,jmh_email_validation_cols).withColumn("vertical",lit("Jumeirah"))
Write_DataFrame_To_Blob(jmh_invalid_email_df,invalid_records_blob_location,'append')
#display(jmh_invalid_email_df)
####Write Invalid Invalid email Format Records of Customer Details to CosmosDB File
#jmh_invalid_email_df.write.format("com.microsoft.azure.cosmosdb.spark").mode("append").options(**writeConfig).save()
#jmh_invalid_email_df.registerTempTable('jmh_invalid_email_df')

In [0]:
%sql
select Error_desc,count(*) from jmh_invalid_email_df group by Error_desc

In [0]:
###Write Invalid Null of both Email & Mobile  Records of Customer Details to Error File
individual_jmh_customer_email_mobile_cols = dq_config['individual_jmh_customer_email_mobile_cols']
jmh_null_invalid_email_mobile_df = Get_Null_Invalid_Email_Mobile_Records(jmh_customer_df,individual_jmh_customer_email_mobile_cols).withColumn("vertical",lit("Jumeirah"))
#Write_DataFrame_To_Blob(jmh_null_invalid_email_mobile_df,invalid_records_blob_location,'append')
display(jmh_null_invalid_email_mobile_df)
####Write Invalid Null of both Email & Mobile  Records of Customer Details to CosmosDB File
#jmh_null_invalid_email_mobile_df.write.format("com.microsoft.azure.cosmosdb.spark").mode("append").options(**writeConfig).save()
#jmh_null_invalid_email_mobile_df.registerTempTable('jmh_null_invalid_email_mobile_df')

In [0]:
%sql
select Error_dec,count(*) from jmh_null_invalid_email_mobile_df group by Error_dec

In [0]:
####Get the Valid Numeric Records of Customer Details
jmh_customer_numeric_validation_cols = dq_config['jmh_customer_numeric_validation_cols']
jmh_valid_numeric_df = Get_Valid_Numeric_Records(jmh_customer_df,jmh_customer_numeric_validation_cols)
#display(jmh_valid_numeric_df)

In [0]:
####Get the Valid Date Records of Customer Details for ValidNumericRecors
jmh_date_validation_cols = dq_config['jmh_customer_date_validation_cols']
jmh_valid_date_df = Get_Valid_DateFormat_Records(jmh_valid_numeric_df,jmh_date_validation_cols)
#display(jmh_valid_date_df)

In [0]:
####Get the Valid EmailFormat  Records of Customer Details for Valid NumericRecors&DateFormat records
jmh_email_validation_cols = dq_config['jmh_customer_email_validation_cols']
jmh_valid_email_df = Get_Valid_EmailFormat_Records(jmh_valid_date_df,jmh_email_validation_cols)
#display(jmh_valid_email_df)

In [0]:
####Get the valid  records either email or mobile_number is notnull for NumericRecors,DateFormat&Email format records
individual_jmh_customer_email_mobile_cols = dq_config['individual_jmh_customer_email_mobile_cols']
jmh_not_null_valid_email_mobile_df = Get_NotNull_Valid_Email_Mobile_Records(jmh_valid_email_df,individual_jmh_customer_email_mobile_cols)
#display(jmh_not_null_valid_email_mobile_df)

In [0]:
####Get the NotNull Valid Records for the Business Key Columns
individual_jmh_customer_key_cols = dq_config['individual_jmh_customer_key_cols']
jmh_not_null_valid_df = Get_Not_Null_Valid_Records(jmh_not_null_valid_email_mobile_df,individual_jmh_customer_key_cols)
Write_DataFrame_To_Blob(jmh_not_null_valid_df,valid_records_blob_location)
#display(jmh_not_null_valid_df)

In [0]:
####Get the Null Invalid Records for Business Key Columns
individual_jmh_customer_key_cols = dq_config['individual_jmh_customer_key_cols']
jmh_invalid_null_df = Append_Error_Message(jmh_not_null_valid_email_mobile_df,'individual_jmh_customer_key_cols',individual_jmh_customer_key_cols).withColumn("vertical",lit("Jumeirah"))
# Write_DataFrame_To_Blob(jmh_invalid_null_df,invalid_records_blob_location,'append')
# #display(jmh_invalid_null_df)
# ####Write the Null Invalid Records of Customer to CosmosDB File
# jmh_invalid_null_df.write.format("com.microsoft.azure.cosmosdb.spark").mode("append").options(**writeConfig).save()
#display(jmh_invalid_null_df)
jmh_invalid_null_df.registerTempTable('jmh_invalid_null_df')

In [0]:
%sql
select Error_desc,count(*) from jmh_invalid_null_df group by Error_desc